In [22]:
%env PYTORCH_ENABLE_MPS_FALLBACK=1
## GPU 없으면 CPU로 돌아가게끔 설정

env: PYTORCH_ENABLE_MPS_FALLBACK=1


In [23]:
import pandas as pd
import matplotlib.pyplot as plt

from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from gluonts.torch import DeepAREstimator


In [24]:
# Load data from a CSV file into a PandasDataset
df = pd.read_csv(
    'data/merged/merged_S&P 500 Financials.csv',
    index_col=0,
    parse_dates=True,
)

In [25]:
duplicate_rows = df[df.duplicated(keep=False)]
df = df.drop_duplicates()
# Ensure uniformly spaced index by reindexing
all_times = pd.date_range(start=df.index.min(), end=df.index.max(), freq='D')
df = df.reindex(all_times)

df.interpolate(method='linear')

print("\nDataFrame with Averaged Missing Values (Linear Interpolation):")
print(df)


DataFrame with Averaged Missing Values (Linear Interpolation):
           bitcoin_time  bitcoin_high  bitcoin_low  bitcoin_open  \
2018-07-02     2018.7.2       6673.71      5537.86       6342.69   
2018-07-03     2018.7.3       6670.30      6473.48       6616.60   
2018-07-04          NaN           NaN          NaN           NaN   
2018-07-05     2018.7.5       6691.32      6448.23       6589.77   
2018-07-06     2018.7.6       6633.44      6458.14       6534.81   
...                 ...           ...          ...           ...   
2024-05-13    2024.5.13      63445.72     60758.38      61459.22   
2024-05-14    2024.5.14      63097.07     61094.77      62932.29   
2024-05-15    2024.5.15      66467.29     61325.48      61548.48   
2024-05-16    2024.5.16      66760.54     64600.05      66245.16   
2024-05-17    2024.5.17      67489.24     65120.39      65250.53   

            bitcoin_volumefrom  bitcoin_volumeto  bitcoin_close  \
2018-07-02            73069.19      4.659157e+08    

/var/folders/9d/xf5zmlhd1ssbgpgvdsc55t940000gn/T/ipykernel_35710/2211150860.py:7: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df.interpolate(method='linear')


In [26]:

dataset = PandasDataset(df, timestamp='Date', freq='D',target="S&P 500 Financials.csv_Open")

# Split the data for training and testing
training_data, test_gen = split(dataset,date=pd.Period('2021-01-01'))
test_data = test_gen.generate_instances(prediction_length=12, windows=3)


In [27]:

# Train the model and make predictions
model = DeepAREstimator(
    prediction_length=12, freq="D", trainer_kwargs={"max_epochs": 5}
).train(training_data)

forecasts = list(model.predict(test_data.input))


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/namjihyeon/.pyenv/versions/3.10.13/lib/python3.10/site-packages/lightning/pytorch/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.


NotImplementedError: The operator 'aten::_standard_gamma' is not currently implemented for the MPS device. If you want this op to be added in priority during the prototype phase of this feature, please comment on https://github.com/pytorch/pytorch/issues/77764. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.